In [61]:
import sys
import importlib

import numpy as np
import pandas as pd
import xarray as xr
import pvlib as pv

In [24]:
sys.path.append('/home/travis/python_code/letkf_forecasting/')
import letkf_forecasting as letkf

In [53]:
importlib.reload(letkf)

<module 'letkf_forecasting' from '/home/travis/python_code/letkf_forecasting/letkf_forecasting.py'>

In [28]:
#for Golem
sat_14 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/sat_14.nc')
sat_15 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/sat_15.nc')
wind_15 = xr.open_dataset('/home/travis/python_code/forecasting/current_data/wind_15_crop.nc')
sensor_data = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_data.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_loc.h5')

In [44]:
sensor_loc2 = np.concatenate([sensor_loc['lat'].values[:, None],
                              sensor_loc['lon'].values[:, None]],
                             axis=1)
sat_loc = np.concatenate([sat_15.lat.values.ravel()[:, None],
                          sat_15.long.values.ravel()[:, None]],
                         axis=1)

In [55]:
sensor_loc3, H = letkf.forward_obs_mat(sensor_loc2, sat_loc)

(27, 3)


In [80]:
j, i = np.meshgrid([0,2,4], [0,2,4])
print(i)
print(j)
this = np.arange(6**2).reshape(6,6)
print(this)
print(this[j, i])

[[0 0 0]
 [2 2 2]
 [4 4 4]]
[[0 2 4]
 [0 2 4]
 [0 2 4]]
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]
[[ 0 12 24]
 [ 2 14 26]
 [ 4 16 28]]


In [68]:
solar_position

,apparent_elevation,apparent_zenith,azimuth,elevation,equation_of_time,zenith
2014-04-15 08:30:00-07:00,31.830799,58.169201,98.727781,31.805954,0.000024,58.194046


In [69]:
#This is taken from http://www.groundcontrol.com/Satellite_Look_Angle_Calculator.htm
goes15_azimuth = 220.5*(2*np.pi)/360
goes15_elevation = 44.1*(2*np.pi)/360

time = pd.Timestamp('2014-04-15 12:00:00').tz_localize('MST')
tus = pv.location.Location(32.2, -111, 'US/Arizona', 700,'Tucson')
solar_position = tus.get_solarposition(time)
letkf.parallax_shift(6500,
                     goes15_azimuth,
                     goes15_elevation,
                     solar_position['azimuth'].values,
                     solar_position['elevation'].values)

(array([ 7672.68778907]), array([ 5881.07338742]))